# Connect to MLflow in Databricks

In [ ]:
import mlflow
import mlflow.tracking._model_registry.utils
from dotenv import load_dotenv

load_dotenv()

# The login and mlflow model registery are set this way because of local development. If you are running this in Databricks, you can remove the login and use the regualr mlflow registry.

mlflow.login()

mlflow.tracking._model_registry.utils._get_registry_uri_from_spark_session = (
    lambda: "databricks-uc"
)

mlflow.set_experiment("/BusinessAutomation")

# Register the model as code

In [ ]:
from mlflow.models import infer_signature

model_input_example = "How did ETL work in the first generation platforms?"

model_output_example = {
    "question": "How did ETL work in the first generation platforms?",
    "answer": "In first generation data platforms, ETL (Extract, Transform, Load) processes were typically batch-oriented and relied heavily on manual coding. Data was extracted from source systems, transformed using custom scripts or specialized ETL tools, and then loaded into data warehouses. These processes were often rigid, time-consuming, and required significant maintenance overhead.",
    "context": [
        {
            "corpus_id": "doc123",
            "id": "section_5.2",
            "text": "First generation data platforms relied on batch ETL processes that extracted data from operational systems during off-hours, transformed it using custom code, and loaded it into data warehouses for analysis.",
            "score": 0.95,
        },
        {
            "corpus_id": "doc456",
            "id": "section_3.7",
            "text": "Traditional ETL workflows in early data platforms were characterized by rigid schedules, limited scalability, and high maintenance costs compared to modern approaches.",
            "score": 0.87,
        },
    ],
}

signature = infer_signature(model_input_example, model_output_example)
signature

In [ ]:
with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(
        artifact_path="model",
        streamable=True,
        pip_requirements="../../requirements.txt",
        python_model="BusinessAutomation.py",
        signature=signature,
        input_example=model_input_example,
    )

my_model = mlflow.pyfunc.load_model(model_info.model_uri)

# Test the registered model

In [ ]:
my_model.predict(["How did ETL work in the first generation platforms?", "hi"])